# Testing Adafruit IO API
#### https://adafruit-io-python-client.readthedocs.io/en/latest/index.html

In [62]:
import datetime
from Adafruit_IO import *

In [84]:
import pandas as pd

In [63]:
dStamp=datetime.datetime.now().strftime("%m_%d_%Y")
hashPath = '/Users/cad/simpHashes/csIO.txt'

In [64]:
def connect_REST(hashPath):
    simpHash=open(hashPath)
    a=list(simpHash)
    userName = a[0].strip()
    apiKey = a[1]
    restClient = Client(userName,apiKey)
    return restClient
    
def connect_MQTT(hashPath):
    simpHash=open(hashPath)
    a=list(simpHash)
    userName = a[0].strip()
    apiKey = a[1]
    mqttClient = MQTTClient(userName,apiKey)
    return mqttClient

In [65]:
aio=connect_REST(hashPath)

In [66]:
# check all the feeds
# This will make a list of all feeds.
allFeeds = aio.feeds()

In [67]:
print(allFeeds)

[Feed(name='rig-cadbook', key='rig-cadbook', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-rig', key='cad-rig', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-waterconsumed', key='cad-waterconsumed', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-topvol', key='cad-topvol', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-weight', key='cad-weight', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad2-rig', key='cad2-rig', description=None, unit_typ

In [68]:
feedList = []
for things in allFeeds:
    feedList.append(things.name)
print(feedList)

['rig-cadbook', 'cad-rig', 'cad-waterconsumed', 'cad-topvol', 'cad-weight', 'cad2-rig', 'cad2-waterconsumed', 'cad2-topvol', 'cad2-weight', 'newFeed']


In [40]:
# let's see what a feed's metadata contains.
print(allFeeds[1])

Feed(name='cad-rig', key='cad-rig', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


#### The name is needed to create the feed, but the key is how it is addressed later. By default, the API will respect formating of text, but not the key. So, keep this in mind. 

In [41]:
thisFeed = aio.feeds('cad-rig')
print(thisFeed)

Feed(name='cad-rig', key='cad-rig', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


In [42]:
# lets create a feed with an uppercase char, this is ok
feedName = "asciiFeed"
feed = Feed(feedName)
createResult = aio.create_feed(feed)

In [43]:
print(createResult)

Feed(name='asciiFeed', key='asciifeed', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


In [45]:
# but to access it, we have to ask for lowercase only, because note above the key is lowercase
thisFeed = aio.feeds(feedName.lower())
print(thisFeed)

Feed(name='asciiFeed', key='asciifeed', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


In [46]:
# now let's delete the feed
feed = aio.delete_feed(feedName.lower())

In [69]:
# now let's get a feed's data
allData = aio.data('cad-waterconsumed')

In [70]:
allData

[Data(created_epoch=1541074185, created_at='2018-11-01T12:09:45Z', updated_at=None, value='0.0023', completed_at=None, feed_id=900018, expiration='2018-12-31T12:09:45Z', position=None, id='0E0GBWPMZ6P21VPNKV1KB4Q75N', lat=None, lon=None, ele=None),
 Data(created_epoch=1541072640, created_at='2018-11-01T11:44:00Z', updated_at=None, value='0.1191', completed_at=None, feed_id=900018, expiration='2018-12-31T11:44:00Z', position=None, id='0E0GBDZ583SP522Y1YN2QCS1P0', lat=None, lon=None, ele=None),
 Data(created_epoch=1541072481, created_at='2018-11-01T11:41:21Z', updated_at=None, value='0.1123', completed_at=None, feed_id=900018, expiration='2018-12-31T11:41:21Z', position=None, id='0E0GBCEJ0T5YKBX8YEEB8CHYAW', lat=None, lon=None, ele=None),
 Data(created_epoch=1541072315, created_at='2018-11-01T11:38:35Z', updated_at=None, value='0.1077', completed_at=None, feed_id=900018, expiration='2018-12-31T11:38:35Z', position=None, id='0E0GBAVZWN6CSAX40V3Q3TDS92', lat=None, lon=None, ele=None),
 Dat

In [99]:
curFeedName = 'cad-waterconsumed'
tCreated_ats=[]
tDates=[]
tTimes=[]
tIDs = []
tValues=[]
tFeedNames = []
for x in allData:
    tCreated_ats.append(x.created_at)
    tS=x.created_at.split('T')
    tDates.append(tS[0])
    tTimes.append(tS[1])
    tIDs.append(x.id)
    tValues.append(float(x.value))
    tFeedNames.append(curFeedName)

tempArray = [tFeedNames,tCreated_ats,tDates,tTimes,tIDs,tValues]
varIndex = ['feed_name','created_at','date','time','id','value']
tempArray=list(list(zip(*tempArray)))
newDF = pd.DataFrame(tempArray,columns = varIndex)
newDF.to_csv('/Users/cad/testFEEDBU.csv')

In [96]:
newDF

,feed_name,created_at,date,time,id,value
0,cad-waterconsumed,2018-11-01T12:09:45Z,2018-11-01,12:09:45Z,0E0GBWPMZ6P21VPNKV1KB4Q75N,0.0023
1,cad-waterconsumed,2018-11-01T11:44:00Z,2018-11-01,11:44:00Z,0E0GBDZ583SP522Y1YN2QCS1P0,0.1191
2,cad-waterconsumed,2018-11-01T11:41:21Z,2018-11-01,11:41:21Z,0E0GBCEJ0T5YKBX8YEEB8CHYAW,0.1123
3,cad-waterconsumed,2018-11-01T11:38:35Z,2018-11-01,11:38:35Z,0E0GBAVZWN6CSAX40V3Q3TDS92,0.1077
4,cad-waterconsumed,2018-11-01T11:36:42Z,2018-11-01,11:36:42Z,0E0GB9SDRPR416W374WH10H8BS,0.1031
5,cad-waterconsumed,2018-11-01T00:56:17Z,2018-11-01,00:56:17Z,0E0FZVAY072EEA2J9HFCWAVAD1,0.0894
6,cad-waterconsumed,2018-11-01T00:41:16Z,2018-11-01,00:41:16Z,0E0FZJQYM5PGTEX4666RPASBH9,0.0504
7,cad-waterconsumed,2018-11-01T00:33:13Z,2018-11-01,00:33:13Z,0E0FZE4HY8KM3XJD29Z4QMH7GM,0.0436
8,cad-waterconsumed,2018-11-01T00:32:19Z,2018-11-01,00:32:19Z,0E0FZDM16JA5FRSZJ37TJTCY68,0.0390
9,cad-waterconsumed,2018-11-01T00:31:13Z,2018-11-01,00:31:13Z,0E0FZD02N9EKWF56N24QN9MB6T,0.0390


In [88]:
list(list(zip(*tempArray)))

[('2018-11-01T12:09:45Z',
  '2018-11-01',
  '12:09:45Z',
  '0E0GBWPMZ6P21VPNKV1KB4Q75N',
  0.0023),
 ('2018-11-01T11:44:00Z',
  '2018-11-01',
  '11:44:00Z',
  '0E0GBDZ583SP522Y1YN2QCS1P0',
  0.1191),
 ('2018-11-01T11:41:21Z',
  '2018-11-01',
  '11:41:21Z',
  '0E0GBCEJ0T5YKBX8YEEB8CHYAW',
  0.1123),
 ('2018-11-01T11:38:35Z',
  '2018-11-01',
  '11:38:35Z',
  '0E0GBAVZWN6CSAX40V3Q3TDS92',
  0.1077),
 ('2018-11-01T11:36:42Z',
  '2018-11-01',
  '11:36:42Z',
  '0E0GB9SDRPR416W374WH10H8BS',
  0.1031),
 ('2018-11-01T00:56:17Z',
  '2018-11-01',
  '00:56:17Z',
  '0E0FZVAY072EEA2J9HFCWAVAD1',
  0.0894),
 ('2018-11-01T00:41:16Z',
  '2018-11-01',
  '00:41:16Z',
  '0E0FZJQYM5PGTEX4666RPASBH9',
  0.0504),
 ('2018-11-01T00:33:13Z',
  '2018-11-01',
  '00:33:13Z',
  '0E0FZE4HY8KM3XJD29Z4QMH7GM',
  0.0436),
 ('2018-11-01T00:32:19Z',
  '2018-11-01',
  '00:32:19Z',
  '0E0FZDM16JA5FRSZJ37TJTCY68',
  0.039),
 ('2018-11-01T00:31:13Z',
  '2018-11-01',
  '00:31:13Z',
  '0E0FZD02N9EKWF56N24QN9MB6T',
  0.039),
 (

In [73]:
print(a.sort())

None


In [75]:
type(allData.)

list

In [55]:
testValues = []
for x in allData:
    testValues.append(float(x.value))

In [56]:
testValues

[0.0023,
 0.1191,
 0.1123,
 0.1077,
 0.1031,
 0.0894,
 0.0504,
 0.0436,
 0.039,
 0.039,
 0.0368,
 0.0368,
 0.0368,
 0.0299,
 0.1515,
 0.1217,
 0.039,
 0.039,
 0.039,
 0.0276,
 0.0,
 0.0069,
 0.0276,
 0.0046,
 0.0023,
 0.0161,
 0.0253,
 0.0322,
 0.0322,
 0.1747,
 0.0712,
 0.0184,
 0.1908,
 0.1033,
 0.0758,
 0.0,
 0.0023,
 0.0023,
 0.0253,
 0.0138,
 0.0138,
 0.0,
 0.0,
 0.0,
 0.1606,
 0.1606,
 0.1606,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1538,
 0.1447,
 0.1332,
 0.0873,
 0.0322,
 0.023,
 0.023,
 0.0115,
 0.0184,
 0.0]

In [61]:
# this is how we look at feeds containing something (simple list enumeration)
for idx, val in enumerate(feedList):
    if 'weight' in val:
        print(idx)

4
8
